In [ ]:
import numpy as np
import cv2

In [ ]:
def to_grayscale(image):
    """
    使用 OpenCV 将彩色图像转换为灰度图像
    :param image: 输入的彩色图像 (H, W, 3)
    :return: 灰度图像 (H, W)
    """
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
# 用高斯滤波器平滑图像
def smooth(image, sigma = 1.4, length = 5):
    """
    sigma:高斯滤波器标准差，控制平滑程度
    length:高斯滤波器大小
    """
    # Compute gaussian filter
    k = length // 2
    gaussian = np.zeros([length, length]) # length × length 二维数组, 存储高斯滤波器的权重
    for i in range(length):
        for j in range(length): 
            # (i-k) 和 (j-k) 表示当前点到滤波器中心的距离
            gaussian[i, j] = np.exp(-((i-k) ** 2 + (j-k) ** 2) / (2 * sigma ** 2))
    gaussian /= 2 * np.pi * sigma ** 2
    # Batch Normalization
    gaussian = gaussian / np.sum(gaussian) # 使滤波器的所有权重之和为 1: 为了保证滤波器不会改变图像的整体亮度

    # Use Gaussian Filter
    W, H = image.shape
    new_image = np.zeros([W - k * 2, H - k * 2]) # 直接裁剪图片

    for i in range(W - 2 * k):
        for j in range(H - 2 * k):
            new_image[i, j] = np.sum(image[i:i+length, j:j+length] * gaussian)

    new_image = np.uint8(new_image) # 0-255, 超出范围就截断(取模)
    return new_image

In [ ]:
# 计算梯度幅值和方向
def get_gradient_and_direction(image):
    Gx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    Gy = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

    W, H = image.shape
    gradients = np.zeros([W - 2, H - 2])
    direction = np.zeros([W - 2, H - 2])

    for i in range(W - 2):
        for j in range(H - 2):
            dx = np.sum(image[i:i+3, j:j+3] * Gx)
            dy = np.sum(image[i:i+3, j:j+3] * Gy)
            gradients[i, j] = np.sqrt(dx ** 2 + dy ** 2)
            if dx == 0:
                direction[i, j] = np.pi / 2
            else:
                direction[i, j] = np.arctan(dy / dx)

    gradients = np.uint8(gradients)

    return gradients, direction

In [ ]:
# 对梯度幅值进行非极大值抑制
def NMS(gradients, direction):
    """
    根据梯度方向，比较当前像素点与其梯度方向上的相邻像素点的梯度幅值。
    如果当前像素点的梯度幅值不是局部最大值，则将其置为 0。
    """
    W, H = gradients.shape
    # 裁剪掉了边界的 1 像素（gradients[1:-1, 1:-1]），因为边界像素无法进行完整的梯度方向比较
    nms = np.copy(gradients[1:-1, 1:-1]) # 存储非极大值抑制后的结果

    for i in range(1, W - 1):
        for j in range(1, H - 1):
            theta = direction[i, j]
            weight = np.tan(theta) # 计算梯度方向的斜率，用于插值计算。
            # 根据梯度方向 theta，选择当前像素点梯度方向上的两个相邻像素点：
              # d1 和 d2 表示梯度方向上的两个像素点的相对位置。
              # 梯度方向被分为 4 个区间：
                # 垂直方向（theta > π/4）：比较上下两个像素。
                # 对角方向（0 ≤ theta ≤ π/4）：比较对角线方向的像素。
                # 水平方向（-π/4 ≤ theta < 0）：比较左右两个像素。
                # 反对角方向（theta < -π/4）：比较反对角线方向的像素。
            
            if theta > np.pi / 4:
                d1 = [0, 1]
                d2 = [1, 1]
                weight = 1 / weight
            elif theta >= 0:
                d1 = [1, 0]
                d2 = [1, 1]
            elif theta >= - np.pi / 4:
                d1 = [1, 0]
                d2 = [1, -1]
                weight *= -1
            else:
                d1 = [0, -1]
                d2 = [1, -1]
                weight = -1 / weight

            g1 = gradients[i + d1[0], j + d1[1]]
            g2 = gradients[i + d2[0], j + d2[1]]
            g3 = gradients[i - d1[0], j - d1[1]]
            g4 = gradients[i - d2[0], j - d2[1]]

            grade_count1 = g1 * weight + g2 * (1 - weight)
            grade_count2 = g3 * weight + g4 * (1 - weight)

            if grade_count1 > gradients[i, j] or grade_count2 > gradients[i, j]:
                nms[i - 1, j - 1] = 0
            
            return nms


In [ ]:
# 用双阈值算法检测
def double_threshold(nms, threshold1, threshold2):
    visited = np.zeros_like(nms)
    output_image = nms.copy()
    W, H = output_image.shape

    def dfs(i, j):
        if i >= W or i < 0 or j >= H or j < 0 or visited[i, j] == 1:
            return
        visited[i, j] = 1
        if output_image[i, j] > threshold1: # 梯度值大于 threshold1，将其标记为强边缘（255）
            output_image[i, j] = 255
            # 递归检查(i, j)的 8 邻域的像素点
            dfs(i-1, j-1) # Z 形遍历
            dfs(i-1, j)
            dfs(i-1, j+1)
            dfs(i, j-1)
            dfs(i, j+1)
            dfs(i+1, j-1)
            dfs(i+1, j)
            dfs(i+1, j+1)
        else:
            output_image[i, j] = 0 # 梯度值小于等于 threshold1，将其标记为非边缘（0）

    for w in range(W):
        for h in range(H):
            if visited[w, h] == 1:
                continue
            if output_image[w, h] >= threshold2: # 调用 dfs 将其标记为强边缘，并尝试连接弱边缘
                dfs(w, h)
            elif output_image[w, h] <= threshold1:
                output_image[w, h] = 0
                visited[w, h] = 1

    # 清除未连接的弱边缘
    for w in range(W):
        for h in range(H):
            if visited[w, h] == 0:
                output_image[w, h] = 0 # 如果像素点未访问过，说明它是未连接的弱边缘，将其标记为非边缘（0）

    return output_image

In [ ]:
# 用双阈值算法检测（非递归实现）
def double_threshold_stack(nms, threshold1, threshold2):
    visited = np.zeros_like(nms)
    output_image = nms.copy()
    W, H = output_image.shape

    # 非递归实现的 DFS
    def dfs_non_recursive(start_i, start_j):
        stack = [(start_i, start_j)]  # 使用栈存储需要访问的像素点
        while stack:
            i, j = stack.pop()
            if i < 0 or i >= W or j < 0 or j >= H or visited[i, j] == 1:
                continue
            visited[i, j] = 1
            if output_image[i, j] > threshold1:  # 梯度值大于 threshold1，标记为强边缘
                output_image[i, j] = 255
                # 将 8 邻域的像素点加入栈
                stack.append((i-1, j-1))
                stack.append((i-1, j))
                stack.append((i-1, j+1))
                stack.append((i, j-1))
                stack.append((i, j+1))
                stack.append((i+1, j-1))
                stack.append((i+1, j))
                stack.append((i+1, j+1))
            else:
                output_image[i, j] = 0  # 梯度值小于等于 threshold1，标记为非边缘

    # 遍历图像
    for w in range(W):
        for h in range(H):
            if visited[w, h] == 1:
                continue
            if output_image[w, h] >= threshold2:  # 如果是强边缘，调用非递归 DFS
                dfs_non_recursive(w, h)
            elif output_image[w, h] <= threshold1:  # 如果是非边缘，直接标记为 0
                output_image[w, h] = 0
                visited[w, h] = 1

    # 清除未连接的弱边缘
    for w in range(W):
        for h in range(H):
            if visited[w, h] == 0:
                output_image[w, h] = 0  # 未访问过的像素点标记为非边缘

    return output_image

### 运行

In [ ]:
image = cv2.imread("./imgs/clock016.jpg")
# cv2.imshow("myImg", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
# print(type(image))
# image 

In [ ]:
# print(image.shape)  # 图像的形状 (高度, 宽度, 通道数)

In [ ]:
# 转灰度图像
gray_image = to_grayscale(image)
# print(type(gray_image))
# print(gray_image.shape)
# print(gray_image)

In [ ]:
# 平滑处理
smoothed_image = smooth(gray_image)
# print(type(smoothed_image))
# print(smoothed_image.shape)
# print(smoothed_image)

In [ ]:
# 计算梯度和方向
gradients, direction = get_gradient_and_direction(smoothed_image)

In [ ]:
# print(gradients.shape)
# gradients

In [ ]:
# print(direction.shape)
# direction

In [ ]:
# 非极大值抑制
nms = NMS(gradients, direction)
# print(nms.shape)
# nms

In [ ]:
# 双阈值算法检测

output_image = double_threshold_stack(nms, 40, 100)

In [ ]:
print(output_image.shape)
output_image


In [ ]:
import matplotlib.pyplot as plt

plt.imshow(output_image, cmap='gray')  # 使用灰度图显示
plt.title('Output Image')  # 设置标题
plt.axis('off')  # 隐藏坐标轴
plt.show()  # 显示图像

### 使用OpenCV自带函数

In [ ]:
# OpenCV自带函数 可以进行高斯平滑和canny算子
image = cv2.GaussianBlur(image, (5,5), 0)
canny = cv2.Canny(image, 50, 160)
canny

In [ ]:
plt.imshow(canny, cmap='gray')  # 使用灰度图显示
plt.title('Output Image')  # 设置标题
plt.axis('off')  # 隐藏坐标轴
plt.show()  # 显示图像